## Python's Module, Package and Library

- A module is a simple Python file that contains collections of functions and global variables and with having a .py extension file.
- It is an executable file and to organize all the modules we have the concept called Package in Python.
- A package is a simple directory having collections of modules.

### Reference

- https://www.geeksforgeeks.org/python/what-is-the-difference-between-pythons-module-package-and-library/#

In [6]:
# import the module
import demo_module

demo_module.myModule("Math")

AttributeError: module 'demo_module' has no attribute 'myModule'